# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [28]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module
import yfinance as yf

## Importing Our List of Stocks
Make sure the .csv file is still in your working directory and import it with the following command:

In [29]:
stocks = pd.read_csv('sp_500_stocks.csv')

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [30]:
stock = yf.Ticker('AAPL')
data = stock.info
data

{'address1': 'One Apple Park Way',
 'city': 'Cupertino',
 'state': 'CA',
 'zip': '95014',
 'country': 'United States',
 'phone': '408 996 1010',
 'website': 'https://www.apple.com',
 'industry': 'Consumer Electronics',
 'sector': 'Technology',
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In addition, the company offers various services, such as Apple Arcade, a game subscription service; Apple Fitness+, a personalized

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [31]:
currentPrice = data['currentPrice']
pe_ratio = data.get('trailingPE')
print(currentPrice, pe_ratio)

226.05 34.354103


## Getting PE ratio and current price for the list of stocks & Building Our DataFrame

Just like in our first project, it's now time to execute several API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. 

In [32]:
# Initialize an empty list to store the results
data = []

# Iterate through each stock symbol in the CSV file
for symbol in stocks['Ticker']:
    try:
        # Fetch the stock data using yfinance
        stock = yf.Ticker(symbol)
        stock_info = stock.info

        # Get the stock price and market cap
        stock_price = stock_info.get('currentPrice', None)
        pe_ratio = stock_info.get('trailingPE', None)

        # Append the data to the list
        data.append([symbol, stock_price, pe_ratio, 0])

    except Exception as e:
        print(f"Failed to get data for {symbol}: {e}")
        data.append([symbol, None, None, 0])

# Create a DataFrame from the collected data
data = pd.DataFrame(data, columns= ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy'])

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABC&crumb=cYjiLcOysP.
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABMD?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABMD&crumb=cYjiLcOysP.
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ATVI?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ATVI&crumb=cYjiLcOysP.
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/DISH?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.

In [33]:
data

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,140.04,33.184834,0
1,AAL,10.05,NaN,0
2,AAP,62.19,172.749980,0
3,AAPL,226.05,34.354103,0
4,ABBV,193.90,64.849495,0
...,...,...,...,...
500,YUM,137.40,25.211008,0
501,ZBH,110.65,23.100210,0
502,ZBRA,344.14,77.509010,0
503,ZION,46.59,11.226506,0


In [34]:
data.isna().sum()

Ticker                      0
Price                      42
Price-to-Earnings Ratio    81
Number of Shares to Buy     0
dtype: int64

In [35]:
final_dataframe = data.copy(deep=True)

## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [36]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)

In [37]:
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,APA,29.25,3.165585,0
1,LNC,31.35,3.293067,0
2,UAL,42.07,4.716368,0
3,GM,45.32,5.092135,0
4,APTV,68.93,5.202264,0
5,FMC,63.20,5.401710,0
6,DAL,40.15,5.760402,0
7,SYF,46.62,6.511173,0
8,NOV,17.94,6.744361,0
9,WU,11.79,7.189024,0


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_size` which is the amount the investor has.

In [38]:
portfolio_size = 1000000

You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [39]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,APA,29.25,3.165585,683
1,LNC,31.35,3.293067,637
2,UAL,42.07,4.716368,475
3,GM,45.32,5.092135,441
4,APTV,68.93,5.202264,290
5,FMC,63.20,5.401710,316
6,DAL,40.15,5.760402,498
7,SYF,46.62,6.511173,429
8,NOV,17.94,6.744361,1114
9,WU,11.79,7.189024,1696


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [40]:
import yfinance as yf

# Replace 'AAPL' with the desired ticker symbol
symbol = 'AAPL'
stock = yf.Ticker(symbol)

# Fetch data
stock_info = stock.info
financials = stock.financials
balance_sheet = stock.balance_sheet
cashflow = stock.cashflow

# P/E Ratio
pe_ratio = stock_info.get('trailingPE')

# P/B Ratio
pb_ratio = stock_info.get('priceToBook')

# P/S Ratio
ps_ratio = stock_info.get('priceToSalesTrailing12Months')

# EV/EBITDA
enterprise_value = stock_info.get('enterpriseValue')
ebitda = stock_info.get('ebitda')
ev_to_ebitda = enterprise_value / ebitda if enterprise_value and ebitda else None

# Fetch income statement (annually)
income_statement = stock.financials  # Annually

# Try accessing gross profit directly from the income statement
try:
    gross_profit = income_statement.loc['Gross Profit'].iloc[0]  # Latest annual gross profit
except KeyError:
    gross_profit = None

# Alternative calculation for Gross Profit
if gross_profit is None:
    revenue = stock_info.get('totalRevenue')
    cost_of_revenue = stock_info.get('costOfRevenue')
    gross_profit = revenue - cost_of_revenue if revenue and cost_of_revenue else None

# EV/GP (Gross Profit)
ev_to_gross_profit = enterprise_value / gross_profit if enterprise_value and gross_profit else None

# Print the results
print(f"P/E Ratio: {pe_ratio}")
print(f"P/B Ratio: {pb_ratio}")
print(f"P/S Ratio: {ps_ratio}")
print(f"EV/EBITDA: {ev_to_ebitda}")
print(f"Gross Profit: {gross_profit}")
print(f"EV/GP: {ev_to_gross_profit}")


P/E Ratio: 34.354103
P/B Ratio: 51.586037
P/S Ratio: 8.913019
EV/EBITDA: 26.380117350223305
Gross Profit: 169148000000.0
EV/GP: 20.552405447347883


In [41]:
rv_dataframe = final_dataframe.copy(deep=True)

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [42]:
# Initialize the additional columns with default values
rv_dataframe['Number of Shares to Buy'] = 0
rv_dataframe['PE Percentile'] = None
rv_dataframe['Price-to-Book Ratio'] = None
rv_dataframe['PB Percentile'] = None
rv_dataframe['Price-to-Sales Ratio'] = None
rv_dataframe['PS Percentile'] = None
rv_dataframe['EV/EBITDA'] = None
rv_dataframe['EV/EBITDA Percentile'] = None
rv_dataframe['EV/GP'] = None
rv_dataframe['EV/GP Percentile'] = None
rv_dataframe['RV Score'] = 0

In [43]:
# Fetch and append additional data
for index, row in rv_dataframe.iterrows():
    ticker = row['Ticker']
    try:
        # Fetch the stock data using yfinance
        stock = yf.Ticker(ticker)
        stock_info = stock.info

        # Get the required financial data
        pb_ratio = stock_info.get('priceToBook', None)
        ps_ratio = stock_info.get('priceToSalesTrailing12Months', None)
        enterprise_value = stock_info.get('enterpriseValue', None)
        ebitda = stock_info.get('ebitda', None)

        # EV/EBITDA
        ev_to_ebitda = enterprise_value / ebitda if enterprise_value and ebitda else None

        # Gross Profit Calculation
        income_statement = stock.financials
        try:
            gross_profit = income_statement.loc['Gross Profit'].iloc[0]  # Latest annual gross profit
        except KeyError:
            gross_profit = None

        if gross_profit is None:
            revenue = stock_info.get('totalRevenue', None)
            cost_of_revenue = stock_info.get('costOfRevenue', None)
            gross_profit = revenue - cost_of_revenue if revenue and cost_of_revenue else None

        # EV/GP
        ev_to_gross_profit = enterprise_value / gross_profit if enterprise_value and gross_profit else None

        # Update the DataFrame with the new data
        rv_dataframe.at[index, 'Price-to-Book Ratio'] = pb_ratio
        rv_dataframe.at[index, 'Price-to-Sales Ratio'] = ps_ratio
        rv_dataframe.at[index, 'EV/EBITDA'] = ev_to_ebitda
        rv_dataframe.at[index, 'EV/GP'] = ev_to_gross_profit

    except Exception as e:
        print(f"Failed to get data for {ticker}: {e}")

In [44]:
# Display the updated DataFrame
rv_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,APA,29.25,3.165585,0,None,1.994681,None,1.214607,None,3.429521,None,4.395012,None,0
1,LNC,31.35,3.293067,0,None,0.76626,None,0.392762,None,24.193094,None,None,None,0
2,UAL,42.07,4.716368,0,None,1.314277,None,0.248657,None,4.31198,None,2.160671,None,0
3,GM,45.32,5.092135,0,None,0.726352,None,0.286008,None,8.872457,None,8.334212,None,0
4,APTV,68.93,5.202264,0,None,1.60321,None,0.810397,None,8.06377,None,6.980472,None,0
5,FMC,63.20,5.401710,0,None,1.730227,None,1.931465,None,17.384627,None,6.423102,None,0
6,DAL,40.15,5.760402,0,None,2.078695,None,0.431052,None,5.86922,None,4.698167,None,0
7,SYF,46.62,6.511173,0,None,1.286495,None,2.099292,None,None,None,None,None,0
8,NOV,17.94,6.744361,0,None,1.107271,None,0.793696,None,7.145643,None,4.740372,None,0
9,WU,11.79,7.189024,0,None,9.034483,None,0.933612,None,5.73254,None,3.314077,None,0


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [45]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,APA,29.25,3.165585,0,None,1.994681,None,1.214607,None,3.429521,None,4.395012,None,0
1,LNC,31.35,3.293067,0,None,0.76626,None,0.392762,None,24.193094,None,None,None,0
2,UAL,42.07,4.716368,0,None,1.314277,None,0.248657,None,4.31198,None,2.160671,None,0
3,GM,45.32,5.092135,0,None,0.726352,None,0.286008,None,8.872457,None,8.334212,None,0
4,APTV,68.93,5.202264,0,None,1.60321,None,0.810397,None,8.06377,None,6.980472,None,0
5,FMC,63.20,5.401710,0,None,1.730227,None,1.931465,None,17.384627,None,6.423102,None,0
6,DAL,40.15,5.760402,0,None,2.078695,None,0.431052,None,5.86922,None,4.698167,None,0
7,SYF,46.62,6.511173,0,None,1.286495,None,2.099292,None,None,None,None,None,0
8,NOV,17.94,6.744361,0,None,1.107271,None,0.793696,None,7.145643,None,4.740372,None,0
9,WU,11.79,7.189024,0,None,9.034483,None,0.933612,None,5.73254,None,3.314077,None,0


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [46]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

/var/folders/7w/mm9ktp7n75dgn8rjth6_zv_m0000gn/T/ipykernel_69238/2238724823.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)
/var/folders/7w/mm9ktp7n75dgn8rjth6_zv_m0000gn/T/ipykernel_69238/2238724823.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downca

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [47]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,APA,29.25,3.165585,0,None,1.994681,None,1.214607,None,3.429521,None,4.395012,None,0
1,LNC,31.35,3.293067,0,None,0.766260,None,0.392762,None,24.193094,None,5.830734,None,0
2,UAL,42.07,4.716368,0,None,1.314277,None,0.248657,None,4.311980,None,2.160671,None,0
3,GM,45.32,5.092135,0,None,0.726352,None,0.286008,None,8.872457,None,8.334212,None,0
4,APTV,68.93,5.202264,0,None,1.603210,None,0.810397,None,8.063770,None,6.980472,None,0
5,FMC,63.20,5.401710,0,None,1.730227,None,1.931465,None,17.384627,None,6.423102,None,0
6,DAL,40.15,5.760402,0,None,2.078695,None,0.431052,None,5.869220,None,4.698167,None,0
7,SYF,46.62,6.511173,0,None,1.286495,None,2.099292,None,8.406129,None,5.830734,None,0
8,NOV,17.94,6.744361,0,None,1.107271,None,0.793696,None,7.145643,None,4.740372,None,0
9,WU,11.79,7.189024,0,None,9.034483,None,0.933612,None,5.732540,None,3.314077,None,0


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [48]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])   

0     0.02
1     0.04
2     0.06
3     0.08
4      0.1
5     0.12
6     0.14
7     0.16
8     0.18
9      0.2
10    0.22
11    0.24
12    0.26
13    0.28
14     0.3
15    0.32
16    0.34
17    0.36
18    0.38
19     0.4
20    0.42
21    0.44
22    0.46
23    0.48
24     0.5
25    0.52
26    0.54
27    0.56
28    0.58
29     0.6
30    0.62
31    0.64
32    0.66
33    0.68
34     0.7
35    0.72
36    0.74
37    0.76
38    0.78
39     0.8
40    0.82
41    0.84
42    0.86
43    0.88
44     0.9
45    0.92
46    0.94
47    0.96
48    0.98
49     1.0
Name: PE Percentile, dtype: object
0     0.64
1     0.06
2     0.32
3     0.04
4     0.42
5      0.5
6      0.7
7      0.3
8      0.2
9      1.0
10     0.6
11    0.14
12    0.76
13    0.02
14    0.58
15    0.46
16    0.67
17    0.18
18    0.98
19     0.8
20    0.88
21     0.4
22    0.24
23    0.67
24    0.08
25    0.82
26    0.48
27    0.62
28    0.86
29    0.12
30    0.84
31    0.44
32    0.56
33    0.96
34     0.1
35    0.36
36    0.78
37    0.

In [49]:
rv_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,APA,29.25,3.165585,0,0.02,1.994681,0.64,1.214607,0.52,3.429521,0.02,4.395012,0.3,0
1,LNC,31.35,3.293067,0,0.04,0.766260,0.06,0.392762,0.18,24.193094,1.0,5.830734,0.56,0
2,UAL,42.07,4.716368,0,0.06,1.314277,0.32,0.248657,0.08,4.311980,0.04,2.160671,0.06,0
3,GM,45.32,5.092135,0,0.08,0.726352,0.04,0.286008,0.1,8.872457,0.74,8.334212,0.9,0
4,APTV,68.93,5.202264,0,0.1,1.603210,0.42,0.810397,0.34,8.063770,0.6,6.980472,0.78,0
5,FMC,63.20,5.401710,0,0.12,1.730227,0.5,1.931465,0.78,17.384627,0.98,6.423102,0.76,0
6,DAL,40.15,5.760402,0,0.14,2.078695,0.7,0.431052,0.22,5.869220,0.18,4.698167,0.32,0
7,SYF,46.62,6.511173,0,0.16,1.286495,0.3,2.099292,0.82,8.406129,0.67,5.830734,0.56,0
8,NOV,17.94,6.744361,0,0.18,1.107271,0.2,0.793696,0.3,7.145643,0.4,4.740372,0.34,0
9,WU,11.79,7.189024,0,0.2,9.034483,1.0,0.933612,0.4,5.732540,0.16,3.314077,0.14,0


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [50]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

/var/folders/7w/mm9ktp7n75dgn8rjth6_zv_m0000gn/T/ipykernel_69238/3791486057.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.3' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,APA,29.25,3.165585,0,0.02,1.994681,0.64,1.214607,0.52,3.429521,0.02,4.395012,0.3,0.300
1,LNC,31.35,3.293067,0,0.04,0.766260,0.06,0.392762,0.18,24.193094,1.0,5.830734,0.56,0.368
2,UAL,42.07,4.716368,0,0.06,1.314277,0.32,0.248657,0.08,4.311980,0.04,2.160671,0.06,0.112
3,GM,45.32,5.092135,0,0.08,0.726352,0.04,0.286008,0.1,8.872457,0.74,8.334212,0.9,0.372
4,APTV,68.93,5.202264,0,0.1,1.603210,0.42,0.810397,0.34,8.063770,0.6,6.980472,0.78,0.448
5,FMC,63.20,5.401710,0,0.12,1.730227,0.5,1.931465,0.78,17.384627,0.98,6.423102,0.76,0.628
6,DAL,40.15,5.760402,0,0.14,2.078695,0.7,0.431052,0.22,5.869220,0.18,4.698167,0.32,0.312
7,SYF,46.62,6.511173,0,0.16,1.286495,0.3,2.099292,0.82,8.406129,0.67,5.830734,0.56,0.502
8,NOV,17.94,6.744361,0,0.18,1.107271,0.2,0.793696,0.3,7.145643,0.4,4.740372,0.34,0.284
9,WU,11.79,7.189024,0,0.2,9.034483,1.0,0.933612,0.4,5.732540,0.16,3.314077,0.14,0.380


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [51]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

## Calculating number of shares to buy

In [52]:
portfolio_size = 1000000

In [53]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UAL,42.07,4.716368,475,0.06,1.314277,0.32,0.248657,0.08,4.311980,0.04,2.160671,0.06,0.112
1,KSS,20.45,8.245968,977,0.28,0.595325,0.02,0.131553,0.02,6.836396,0.34,1.382367,0.02,0.136
2,BWA,32.64,9.801802,612,0.46,1.220141,0.24,0.518364,0.24,5.458247,0.14,3.976783,0.18,0.252
3,PVH,99.56,8.881355,200,0.36,1.102559,0.18,0.617109,0.28,7.168469,0.44,1.632794,0.04,0.260
4,VLO,149.82,8.488386,133,0.3,1.886569,0.58,0.357324,0.16,5.205863,0.1,4.368175,0.28,0.284
5,NOV,17.94,6.744361,1114,0.18,1.107271,0.2,0.793696,0.3,7.145643,0.4,4.740372,0.34,0.284
6,TAP,53.63,9.840367,372,0.5,0.927357,0.08,0.931695,0.38,6.698808,0.32,3.923482,0.16,0.288
7,UNM,55.03,8.080764,363,0.24,0.989232,0.14,0.806791,0.32,5.988046,0.2,5.830734,0.56,0.292
8,APA,29.25,3.165585,683,0.02,1.994681,0.64,1.214607,0.52,3.429521,0.02,4.395012,0.3,0.300
9,DAL,40.15,5.760402,498,0.14,2.078695,0.7,0.431052,0.22,5.869220,0.18,4.698167,0.32,0.312


In [54]:
rv_dataframe.to_csv('3.Value Strategy - Stocks to buy.csv')